In [1]:
import pandas as pd
import numpy as np
#Linking to Gdrive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
%cd /content/drive/MyDrive/Work/Formatting_and_data/
#if you want to see the files in your directory use
%ls

Mounted at /content/drive
/content/drive/MyDrive/Work/Formatting_and_data
'Copy of GAN_diabetes.ipynb'   image_at_epoch_0018.png   image_at_epoch_0038.png
 data.xlsx                     image_at_epoch_0019.png   image_at_epoch_0039.png
 GAN_diabetes.ipynb            image_at_epoch_0020.png   image_at_epoch_0040.png
 image_at_epoch_0001.png       image_at_epoch_0021.png   image_at_epoch_0041.png
 image_at_epoch_0002.png       image_at_epoch_0022.png   image_at_epoch_0042.png
 image_at_epoch_0003.png       image_at_epoch_0023.png   image_at_epoch_0043.png
 image_at_epoch_0004.png       image_at_epoch_0024.png   image_at_epoch_0044.png
 image_at_epoch_0005.png       image_at_epoch_0025.png   image_at_epoch_0045.png
 image_at_epoch_0006.png       image_at_epoch_0026.png   image_at_epoch_0046.png
 image_at_epoch_0007.png       image_at_epoch_0027.png   image_at_epoch_0047.png
 image_at_epoch_0008.png       image_at_epoch_0028.png   image_at_epoch_0048.png
 image_at_epoch_0009.png       imag

In [2]:
obs = pd.read_excel('observe_.xlsx')
patients = pd.read_excel('patients_xl.xlsx')
refs = pd.read_excel('var_list_.xlsx')

In [3]:
Unique_patients = (patients["PATIENT"].unique()) #find unique patients within dataset
prediabetes_patients = [] #account for prediabetes then nothing, or diabetes to prediabetes
diabetes_patients = [] #account for first prediabetes diagnosis, then diabetes, or just diabetes
counter = 0
for i in Unique_patients:
  current_patient = i #captures each unique patient within the dataset
  current_patient_list = []
  for j in patients.iterrows():
    if j[1][1] == i:
      current_patient_list.append([i,j[1][2]])
  sorted_list = sorted(current_patient_list, key=lambda x: x[0])
  if sorted_list[-1][1].strip().lower() == 'diabetes':
    #print(sorted_list[-1][1] + " IS THE SAME AS DIABETES: so appened to diabetes")
    diabetes_patients.append(i)
  else:
    prediabetes_patients.append(i)
pre = np.zeros((len(prediabetes_patients),))
de = np.ones((len(diabetes_patients),))
label = np.append(pre,de)
all_patient_tags = prediabetes_patients + diabetes_patients

The goal is to get the data to be formatted with the following:
(60000, 28, 28) (however, we can get away with making is a 364xnx1 since n will be the amount of medical data for the patient
(60000,)

In [5]:
row_select = [0, 1, 2, 4, 5, 6, 7]
ehr_list = refs.iloc[row_select, 2]
ehr_list

0    Pain severity - 0-10 verbal numeric rating [Sc...
1                             Diastolic Blood Pressure
2                              Systolic Blood Pressure
4                                        Urea Nitrogen
5                                           Creatinine
6                                              Calcium
7                                               Sodium
Name: DESCRIPTION, dtype: object

In [ ]:
row_select = [0, 1, 2, 4, 5, 6, 7]
ehr_list = refs.iloc[row_select, 2]
#ehr_list = refs.iloc[0:8, 2]
input_data = np.zeros((len(all_patient_tags),len(ehr_list)))
patient_row_index = 0
for current_patient in all_patient_tags:
  var_col_index = 0
  print("Current Patient: " + str(patient_row_index))
  patient_rows = obs[obs['PATIENT'].str.contains(current_patient)]
  patient_ind = np.array(patient_rows.index)
  for ehr_var in ehr_list:
    #print("   Current variable: " + ehr_var)
    ehr_rows = obs[obs['DESCRIPTION'].str.contains(ehr_var)]
    #print("EHR ROWS")
    #print(ehr_rows)
    #print("________")
    ehr_ind = np.array(ehr_rows.index)
    common_rows = np.intersect1d(patient_ind, ehr_ind)
    #print("STARTSHIT");print(common_rows);print("SHIT");print(ehr_ind);print("STOPSHIT") #OUTPUT OF THIS FIX IT: its doing [], then [167329 1677377 blah blah blah bascially not making a square matrix]
    if common_rows.shape == np.zeros(0).shape:
      input_data[patient_row_index,var_col_index] = np.nan #if no intersection, then don't add data? or make data 0
    else:
      specific_rows = obs.loc[common_rows]
      #print("__");print(specific_rows);print("---")
      dat_to_append = specific_rows.iloc[-1,5] #last measurment since it's sorted by epoch, and 5 col is actual value
      input_data[patient_row_index,var_col_index] = dat_to_append
    var_col_index += 1
  patient_row_index += 1
#pain severity, diastolic, systolic, calcium, sodium, potassium, chloride, body weight (UPPER PART WORKS)

Current Patient: 0
Current Patient: 1


In [ ]:
#process to check if there are any NANs per column: Turn this into a function perhaps later
print(type(input_data))
print(input_data.shape[1])
for i in range(1,input_data.shape[1]):
  count = 0
  for j in range(1,input_data.shape[0]):
    if np.isnan(input_data[j-1, i-1]):
      count += 1
  print("Column: "+ str(i) + " contains: " + str(count) + " NANs")

<class 'numpy.ndarray'>
9
Column: 1 contains: 0 NANs
Column: 2 contains: 0 NANs
Column: 3 contains: 2 NANs
Column: 4 contains: 0 NANs
Column: 5 contains: 0 NANs
Column: 6 contains: 0 NANs
Column: 7 contains: 0 NANs
Column: 8 contains: 0 NANs


In [ ]:
#saving current inputs to work around with:
df = pd.DataFrame(input_data)
df2 = pd.DataFrame(label)
## save to xlsx file
filepath = 'data.xlsx'
df.to_excel(filepath, index=False)
filepath_l = 'label.xlsx'
df2.to_excel(filepath_l, index=False)